In [1]:
import pandas as pd

In [2]:
orig = pd.read_csv('output/edgr_final/carrier_non_carrier_family_corrected.tsv', sep='\t')
orig = orig.set_index('ensembl', drop=False)

pheno = pd.read_csv('../data/pheno_final.tsv', sep='\t')
pheno = pheno.drop_duplicates('subject')
pheno = pheno.set_index('subject', drop=False)

In [3]:
df = pd.read_csv('output/edgr_exclude_one/SG001.tsv', sep='\t')
df['dir'] = df.logFC.apply(lambda x: '+'if x > 0 else '-')
df['gene_dir'] = df.ensembl + df.dir
intersect = df.gene_dir.to_list()

for sub in pheno.subject:
    df = pd.read_csv('output/edgr_exclude_one/{}.tsv'.format(sub), sep='\t')
    df['dir'] = df.logFC.apply(lambda x: '+'if x > 0 else '-')
    df['gene_dir'] = df.ensembl + df.dir
    excl_de = df.gene_dir.to_list()
    intersect = list(set(intersect) & set(excl_de))

In [5]:
keep = pd.read_csv('output/edgr_exclude_one/keep.SG001.txt', sep='\t', header=None)
keep = keep[0].to_list()
for sub in pheno.subject:
    df = pd.read_csv('output/edgr_exclude_one/keep.{}.txt'.format(sub), sep='\t', header=None)
    excl_keep = df[0].to_list()
    keep = list(set(keep) & set(excl_keep))

In [7]:
with open('output/edgr_exclude_one/keep.intersect.txt', 'w') as f:
    for gene in keep:
        f.write(gene)
        f.write('\n')

In [8]:
orig[orig.ensembl.isin([s[:-1] for s in intersect])].to_csv('output/edgr_exclude_one/intersect.tsv', sep='\t', index=False)